In [1]:
base_path = "/content/drive/My Drive/Github/ca-hg-sbl-for-jadce-in-mtc"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from ca_hg_sbl import ca_hg_sbl, build_precision_matrix_aidnc

data = np.load(f'{base_path}/general_model/data/general_model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load(f'{base_path}/communication_model/data/communication_simulation_theta.npz', allow_pickle=True)
all_theta = data["all_theta"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_gamma.npz', allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_z.npz', allow_pickle=True)
all_z = data["all_z"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_noise.npz', allow_pickle=True)
all_noise = data["all_noise"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_received_signal.npz', allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_general.npz', allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

pilot_length_index = 1
pilot_length=pilot_length_array[pilot_length_index]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

beta_array = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

Mounted at /content/drive


In [4]:
sample_range = range(0, 10)
num_samples = len(sample_range)

In [5]:
# Final gamma and z results
gamma_result_all = np.empty((num_samples, len(beta_array)), dtype=object)
z_result_all = np.empty((num_samples, len(beta_array)), dtype=object)

In [6]:
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    z = all_z[sample_index]

    for beta_index in tqdm(range(len(beta_array)), desc="Processing alpha Levels", leave=False, position=1):
        alpha = 1.0
        beta = beta_array[beta_index]

        theta = all_theta[sample_index, pilot_length_index, :pilot_length]
        received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]

        # Compute noise
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
        noise_power = signal_power / snr

        # Compute precision matrix
        precision_matrix = build_precision_matrix_aidnc(loc, alpha=beta, beta=0.1)

        # Run CASBL
        gamma_result, z_result, gamma_history, z_history, iteration_count = ca_hg_sbl(
            theta, received_signal, noise_power, precision_matrix, bounded = True
        )

        # Save outputs
        gamma_result_all[sample_index-sample_range[0], beta_index] = gamma_result.copy()
        z_result_all[sample_index-sample_range[0], beta_index] = z_result.copy()

Processing Samples: 100%|██████████| 10/10 [24:34<00:00, 147.49s/it]


In [7]:
np.savez_compressed(
    f'{base_path}/hyper_parameter_tuning/ca_hg_sbl_anc_b/data/ca_hg_sbl_aidnc_b_beta_all_snr_db_12_pilot_length_25.npz',
    gamma_result_all=gamma_result_all,
    z_result_all=z_result_all,
    beta_array = beta_array,
)

print("All results have been saved")

All results have been saved
